# Challenges:
* Encode companies
* Encode people
* Encode cities and countries
* use text data embedding (word/document) or word freq 
* Imputation

imports

In [1]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
from sklearn import preprocessing
from sklearn.impute import KNNImputer


Reviewing a sample row from each file

In [2]:
acquired = pd.read_csv("Data/Acquired Tech Companies.csv")
acquired.iloc[0]

Company                                                     Day Software
CrunchBase Profile     http://www.crunchbase.com/organization/day-sof...
Image                  http://a5.images.crunchbase.com/image/upload/c...
Tagline                Day Software develops web applications that al...
Year Founded                                                         NaN
Market Categories                                               Software
Address (HQ)           Barfüsserplatz 6, Basel, Basel-Stadt, Switzerland
City (HQ)                                                          Basel
State / Region (HQ)                                          Basel-Stadt
Country (HQ)                                                 Switzerland
Description            Day was founded in Basel, Switzerland, in 1993...
Homepage                                              http://www.day.com
Twitter                                                              NaN
Acquired by                                        

In [3]:
acquiring = pd.read_csv("Data/Acquiring Tech Companies.csv")
acquiring.iloc[0]

Acquiring Company                                                                        Adobe
CrunchBase Profile                               www.crunchbase.com/organization/adobe-systems
Image                                        http://a2.images.crunchbase.com/image/upload/c...
Tagline                                      Adobe is an American multinational computer so...
Market Categories                            Photo Editing, Design, Creative, Software, Ima...
Year Founded                                                                              1982
IPO                                                                                       1986
Founders                                                         John Warnock, Charles Geschke
Number of Employees                                                                     11,144
Number of Employees (year of last update)                                               2012.0
Total Funding ($)                                 

In [4]:
acquisitions = pd.read_csv("Data/Acquisitions.csv")
acquisitions.iloc[0]

Acquisitions ID                                      EMC acquired Data Domain in 2009
Acquired Company                                                          Data Domain
Acquiring Company                                                                 EMC
Year of acquisition announcement                                                 2009
Deal announced on                                                           8/07/2009
Price                                                                  $2,100,000,000
Status                                                                    Undisclosed
Terms                                                                            Cash
Acquisition Profile                 http://www.crunchbase.com/acquisition/5dc676a1...
News                                                         EMC acquired Data Domain
News Link                           http://www.businesswire.com/news/home/20090708...
Name: 0, dtype: object

In [5]:
founders = pd.read_csv("Data/Founders and Board Members.csv")
founders.iloc[0]

Name                                                 Hans-Werner Hector
CrunchBase Profile      http://de.wikipedia.org/wiki/Hans-Werner_Hector
Role                                                            Founder
Companies                                                           SAP
Image                 http://images.forbes.com/media/lists/10/2006/4...
Name: 0, dtype: object

We will link between the files using these columns:
* Acquisitions ID to link the acquisitions
* 'Founders' and 'Name' to link the Founders

In [6]:
np.intersect1d(acquired.columns, acquisitions.columns).tolist()

['Acquisitions ID']

In [7]:
np.intersect1d(acquiring.columns, acquisitions.columns).tolist()

['Acquiring Company', 'Acquisitions ID']

In [8]:
def ValidateLink(url, timeout=15):
    session = requests.Session()
    # fake headers to make it seem like a real request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "DNT": "1",
    }
    session.headers.update(headers)
    try:
        response = session.get(url, timeout=timeout, allow_redirects=True, stream=True)
        status_code = response.status_code
        response.close()
        if status_code < 400:
            return True
        else:
            return False
    except Exception as e:
        return False

In [9]:
def ValidateLinks(urls):
    results = []
    for url in urls:
        results.append(ValidateLink(url))
        if results[-1]:
            return results
    return results

In [10]:
def ValidateLinksDF(df):
    for col in df.columns:
        for val in df[col]:
            if type(val) == str and ("http" in val):
                print(col)
                results = ValidateLinks(df[col])
                if not pd.Series(results).any():
                    print(f'Column "{col}" had no valid links , or is using captcha.')
                    print("Try it yourself:")
                    print(df[col][0] + "\n")
                break

ValidateLinksDF(acquired)

* CrunchBase is using CAPTCHA , so we won't drop it now but we will process it later
* Image links are all corrupt so we will drop the column 

In [11]:
acquired = acquired.drop("Image", axis=1)

ValidateLinksDF(acquiring)

* drop Image also

In [12]:
acquiring = acquiring.drop("Image", axis=1)

ValidateLinksDF(acquisitions)

* acquisitions profile is also a crunchbase link

In [13]:
acquisitions = acquisitions.drop("Acquisition Profile", axis=1)

ValidateLinksDF(founders)

We don't need the exact address of the company, we already have the city , state and country

In [14]:
acquired = acquired.drop("Address (HQ)", axis=1)
acquiring = acquiring.drop("Address (HQ)", axis=1)

**Adding the target variable**

In [15]:
acquisitions["Price"] = [
    int(price.removeprefix("$").replace(",", "")) for price in acquisitions["Price"]
]

In [16]:
acquired["Price"] = None
acquired["Year of acquisition announcement"] = None

In [17]:
for i, company in enumerate(acquisitions["Acquired Company"]):
    acquired.loc[acquired["Company"] == company, "Price"] = acquisitions.iloc[i][
        "Price"
    ]
    acquired.loc[acquired["Company"] == company, "Year of acquisition announcement"] = (
        acquisitions.iloc[i]["Year of acquisition announcement"]
    )

In [18]:
fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

There was a wrongly entered value, so I looked at the link and corrected it

In [19]:
acquisitions.loc[
    acquisitions["Year of acquisition announcement"] == 2104,
    "Year of acquisition announcement",
] = 2014

Plotting again without the error, now we can see that the overall trend of prices tends to go up, that's why we added the 'Year of acquisitions announcement' column

In [20]:
fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=700,
    height=400,
)
fig.show()

update the datatypes automatically

In [21]:
acquired = acquired.infer_objects()
acquisitions = acquisitions.infer_objects()

In [22]:
fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

Another error found and corrected

In [23]:
acquired.loc[acquired["Year Founded"] == 1840, "Year Founded"] = 2006
acquired.loc[acquired["Year Founded"] == 1933, "Year Founded"] = 1989

In [24]:
fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

In [25]:
acquired.iloc[12]["Tagline"]

'5min Media is a syndication platform for lifestyle, knowledge and instructional videos.'

In [26]:
for l in acquired.iloc[12]["Description"].split("."):
    print(l + "\n")

5min Media is the leading syndication platform for lifestyle, knowledge and instructional videos

 Reinventing the cable network online, 5min reaches engaged and targeted audiences of passionate consumers through its network of 100s of lifestyle and niche websites

 The 5min video library comprises more than 200,000 short-form videos from some of the world's largest media companies, as well as the most innovative independent producers

 Visit  for more information





* 'Tagline' contains a brief and precise description of the company , while the 'Description' is very long and doesn't provide any more important details, 
so we will drop the 'Description'

In [27]:
acquiring = acquiring.drop("Description", axis=1)
acquired = acquired.drop("Description", axis=1)

### There isn't any new useful information that we can get out of those , so we will drop them

* "CrunchBase Profile" and "API" columns are both on the crunchbase website , which uses captcha so we can't scrap it, and their API is paid , and the provided API key is invalid , so we can't use it

* "Homepage" column contains the link to the website of every company , and they aren't all the same so we can't apply a function or a program to extract certain information about them. To use the link , this would require us to go over into each of them one by one , which isn't  feasible


* "Twitter" column also can't be scraped according to their new policy , tried multiple APIs and libraries but none of them worked , even twitter's free tier API is useless
 

* "Acquisition ID" is just used to link between files , and we can do that with the company's name


In [28]:
acquired = acquired.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
acquiring = acquiring.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
founders = founders.drop("CrunchBase Profile", axis=1)

In [29]:
acquired["Age on acquisition"] = (
    acquired["Year of acquisition announcement"] - acquired["Year Founded"]
)

In [30]:
acquired = acquired.drop(["Year Founded", "Year of acquisition announcement"], axis=1)

In [31]:
acquired = acquired.astype(
    {
        "Acquired by": "category",
        "City (HQ)": "category",
        "State / Region (HQ)": "category",
        "Country (HQ)": "category",
    }
)

All these columns are probably related to the target column , so we will keep them for now

Market categories contains multiple values , still not processed

In [32]:
acquired.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Company              310 non-null    object  
 1   Tagline              307 non-null    object  
 2   Market Categories    287 non-null    object  
 3   City (HQ)            275 non-null    category
 4   State / Region (HQ)  273 non-null    category
 5   Country (HQ)         276 non-null    category
 6   Acquired by          309 non-null    category
 7   Price                310 non-null    int64   
 8   Age on acquisition   241 non-null    float64 
dtypes: category(4), float64(1), int64(1), object(3)
memory usage: 22.9+ KB


Dropping 'year of last update' of the number of employees , because we don't need it directly and can't use it in any way to pridct the current number

In [33]:
acquiring = acquiring.drop("Number of Employees (year of last update)", axis=1)

There are multiple 'NOT YET' in the IPO column , and the earliest the number the better it is , so we won't replace them with zero ,we will replace them with 2025 or anything larger

In [34]:
acquiring["IPO"].value_counts()[:5]

IPO
1986       4
1978       4
1983       2
Not yet    2
1990       2
Name: count, dtype: int64

In [35]:
acquiring.loc[acquiring["IPO"] == "Not yet", "IPO"] = 2025  # 2025 is debatable
acquiring.loc[acquiring["IPO"].isna(), "IPO"] = 2025  # 2025 is debatable

Idea for acquiring companies: calculate the average price paid for all acquired companies

how to categorize multiple values in the same cell?

In [36]:
acquiring["Market Categories"][:5]

0    Photo Editing, Design, Creative, Software, Ima...
1    Groceries, Consumer Goods, Crowdsourcing, E-Co...
2    News, Advertising Platforms, Content Creators,...
3    Computers, Consumer Electronics, Hardware + So...
4                                               Mobile
Name: Market Categories, dtype: object

In [37]:
acquiring = acquiring.astype(
    {
        "City (HQ)": "category",
        "State / Region (HQ)": "category",
        "Country (HQ)": "category",
        "IPO": "float",
    }
)

In [38]:
flattened = [x for item in acquiring["Board Members"].dropna() for x in item.split(",")]

In [39]:
pd.Series(flattened).nunique()

309

In [40]:
len(np.intersect1d(founders["Name"], flattened))

34

Some of the board members are in the founders df , so we won't drop them for now

In [41]:
acquiring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Acquiring Company       36 non-null     object  
 1   Tagline                 36 non-null     object  
 2   Market Categories       36 non-null     object  
 3   Year Founded            36 non-null     int64   
 4   IPO                     36 non-null     float64 
 5   Founders                36 non-null     object  
 6   Number of Employees     35 non-null     object  
 7   Total Funding ($)       36 non-null     int64   
 8   Number of Acquisitions  36 non-null     int64   
 9   Board Members           34 non-null     object  
 10  City (HQ)               34 non-null     category
 11  State / Region (HQ)     33 non-null     category
 12  Country (HQ)            36 non-null     category
 13  Acquired Companies      36 non-null     object  
dtypes: category(3), float64(1), 

In [42]:
founders["Companies"].value_counts()[:5]

Companies
Microsoft                 21
IBM                       20
Cisco Systems             17
Verizon Communications    15
Nokia                     15
Name: count, dtype: int64

In [43]:
founders["Role"].value_counts()

Role
Board of Directors                    222
Founder                                79
Advisory Board                         72
Board of Directors, Advisory Board      4
Board Observer                          3
Board of Directors, Founder             2
Name: count, dtype: int64

In [44]:
founders = founders.astype({"Role": "category", "Companies": "category"})

The image of the founder doesn't affect anything at all ... DROPPED

In [45]:
founders = founders.drop("Image", axis=1)

Ready

In [46]:
founders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Name       382 non-null    object  
 1   Role       382 non-null    category
 2   Companies  382 non-null    category
dtypes: category(2), object(1)
memory usage: 6.6+ KB


* The specific date which the deal was announced on doesn't matter , what matters is the year so the model can know that inflation affects the price
* The ID doesn't add any new info
* The News and News link don't add any info or details about the acquisition

In [47]:
acquisitions = acquisitions.drop(
    ["Deal announced on", "Acquisitions ID", "News", "News Link"], axis=1
)

In [48]:
acquisitions["Status"].value_counts()

Status
Undisclosed    310
Complete        16
Pending          9
Name: count, dtype: int64

In [49]:
acquisitions["Terms"].value_counts()

Terms
Undisclosed    148
Cash           128
Cash, Stock     36
Stock           24
Name: count, dtype: int64

In [50]:
acquisitions = acquisitions.astype(
    {
        "Terms": "category",
        "Status": "category",
    }
)

In [51]:
acquisitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Acquired Company                  336 non-null    object  
 1   Acquiring Company                 336 non-null    object  
 2   Year of acquisition announcement  336 non-null    int64   
 3   Price                             336 non-null    int64   
 4   Status                            335 non-null    category
 5   Terms                             336 non-null    category
dtypes: category(2), int64(2), object(2)
memory usage: 11.6+ KB


### Spliting each multi-valued category to an array of categories

In [52]:
def SplitMultiValuedColumn(df,label):
    df[label] = [
        [value.lstrip().rstrip() if type(value) == str else value for value in str(values).split(',')] for values in df[label]
    ]
    df[label].info

In [53]:
def getUniqueLabels(df,label):
    uniqueLabels = []
    for labels in df[label]:
        for label in labels:
            if [label] not in uniqueLabels:
                uniqueLabels.append([label])
    return np.ravel(uniqueLabels)

In [54]:
def encodeMultiValuedCategory(df , label : str, categories = []):
    le = preprocessing.LabelEncoder()
    SplitMultiValuedColumn(df,label)
    if len(categories)==0:
        categories = getUniqueLabels(df,label)
    le.fit(np.asarray(categories))
    df[label] = [le.transform(np.asarray(values)) for values in df[label]]
    return le.classes_

def encodeCategory(df , label : str, categories = []):
    le = preprocessing.LabelEncoder()
    if len(categories)==0:
        categories = df[label]
    le.fit(categories)
    df[label] = le.transform(df[label])
    return le.classes_

##### encoding data in Aquisitions

In [55]:
multiValuedLabels = ["Terms","Acquiring Company","Acquired Company"]
for label in multiValuedLabels:
    print(encodeMultiValuedCategory(acquisitions,label)[:5])

['Cash' 'Stock' 'Undisclosed']
['AOL' 'AT&T' 'Adobe' 'Amazon' 'Apple']
['3Com' '3PAR' '5min Media' 'AMD Handheld Graphics'
 'ATG (Art Technology Group)']


In [56]:
acquisitions["Terms"][:5]

0       [0]
1       [0]
2       [2]
3       [2]
4    [0, 1]
Name: Terms, dtype: object

In [57]:
encodeCategory(acquisitions,"Status")

array(['Complete', 'Pending', 'Undisclosed', nan], dtype=object)

In [58]:
acquisitions[:3]

,Acquired Company,Acquiring Company,Year of acquisition announcement,Price,Status,Terms
0,[66],[13],2009,2100000000,2,[0]
1,[220],[0],2007,363000000,2,[0]
2,[208],[8],2008,215000000,2,[2]


##### encoding data in Founders

In [59]:
founders.info

<bound method DataFrame.info of                       Name     Role               Companies
0       Hans-Werner Hector  Founder                     SAP
1              Akio Morita  Founder                    Sony
2    Alexander Graham Bell  Founder                    AT&T
3           Andrew Viterbi  Founder                Qualcomm
4                Bob Miner  Founder                  Oracle
..                     ...      ...                     ...
377              Jack Yuan  Founder                 SanDisk
378        Sanjay Mehrotra  Founder                 SanDisk
379            Jerry Modes  Founder                Teradata
380           David Hartke  Founder                Teradata
381         Charles R. Lee  Founder  Verizon Communications

[382 rows x 3 columns]>

In [60]:
foundersNames = encodeCategory(founders,"Name")

In [61]:
foundersNames[:3]

array(['Adam Grosser', 'Akio Morita', 'Al Gore'], dtype=object)

In [62]:
foundersRoles = encodeMultiValuedCategory(founders,"Role")

In [63]:
foundersRoles

array(['Advisory Board', 'Board Observer', 'Board of Directors',
       'Founder'], dtype='<U18')

##### encoding data in Aquiring

In [64]:
acquiringCompanies = encodeMultiValuedCategory(acquiring,"Acquiring Company")
acquiredCompanies = encodeMultiValuedCategory(acquiring,"Acquired Companies")

categories = ["City (HQ)","State / Region (HQ)","Country (HQ)"]
multiValuedCategories = ["Market Categories","Board Members"]
# saving acquiring companies names so i can encode the same column in different dataFrame with the same values
# and there is multiple shared columns in different dataframes that needs this operation

for c in categories:
    print(encodeCategory(acquiring,c)[:3])

for c in multiValuedCategories:
    print(encodeMultiValuedCategory(acquiring,c)[:3])

['Armonk' 'Cupertino' 'Dallas']
['California' 'Massachusetts' 'New York']
['Canada' 'Germany' 'Norway']
['Advertising Platforms' 'All Markets' 'All Students']
['AJ Jaghori' 'Adam Grosser' 'Al Gore']


In [65]:
encodeMultiValuedCategory(acquiring,"Founders",foundersNames)[:2]

array(['Adam Grosser', 'Akio Morita'], dtype=object)

In [66]:
acquiring[:2]

,Acquiring Company,Tagline,Market Categories,Year Founded,IPO,Founders,Number of Employees,Total Funding ($),Number of Acquisitions,Board Members,City (HQ),State / Region (HQ),Country (HQ),Acquired Companies
0,[2],Adobe is an American multinational computer so...,"[38, 19, 16, 49, 31]",1982,1986.0,"[191, 46]","11,144",0,38,[305],18,0,5,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372..."
1,[3],Amazon is an international e-commerce company ...,"[27, 14, 17, 21]",1994,1997.0,[173],"132,600",8000000,45,"[100, 218, 296, 205, 25, 139, 144, 300]",20,7,5,"[22, 25, 57, 76, 125, 187, 185, 197, 214, 220,..."


##### encoding data in Acquired

In [67]:
encodeCategory(acquired,"Company",acquiredCompanies)

array(['2Web Technologies', '2dehands.be', '2ememain.be', ..., 'threadsy',
       'xkoto', 'zynamics'], dtype='<U36')

In [68]:
# i need to make global variables for this categories to get the unique from merging acquiring and acuired data
# something getUniqueValues(acquired["City (HQ)"] + acquiring["City (HQ)"])

for c in ["City (HQ)","State / Region (HQ)","Country (HQ)","Acquired by"]:
    print(encodeCategory(acquired,c)[:3])
encodeMultiValuedCategory(acquired,"Market Categories")[:3]

['Alameda' 'Aliso Viejo' 'Arcueil']
['Andalucia' 'Arkansas' 'Basel-Stadt']
['Australia' 'Canada' 'China']
['AOL' 'AT&T' 'Adobe']


array(['3D', 'Advertising', 'Aerospace'], dtype='<U27')

In [69]:
acquired[:5]

,Company,Tagline,Market Categories,City (HQ),State / Region (HQ),Country (HQ),Acquired by,Price,Age on acquisition
0,354,Day Software develops web applications that al...,[117],8,2,13,2,240000000,NaN
1,404,"Efficient Frontier, an online performance and ...",[1],101,5,15,2,400000000,9.0
2,751,Macromedia is a graphics and web development s...,[117],91,5,15,2,3400000000,13.0
3,845,Neolane is a marketing technology provider dev...,[117],2,19,5,2,600000000,12.0
4,921,Omniture is an online marketing and web analyt...,"[117, 1]",73,44,15,2,1800000000,13.0


##### Removing the null category in acquisitions.status to test the imputation

In [70]:
acquisitions.isnull().sum()

Acquired Company                    0
Acquiring Company                   0
Year of acquisition announcement    0
Price                               0
Status                              0
Terms                               0
dtype: int64

In [71]:
acquisitions["Status"] = acquisitions["Status"].replace(3, np.nan)


In [72]:
acquisitions.isnull().sum()

Acquired Company                    0
Acquiring Company                   0
Year of acquisition announcement    0
Price                               0
Status                              1
Terms                               0
dtype: int64

In [73]:
acquisitions["Status"].value_counts(dropna=False)

Status
2.0    310
0.0     16
1.0      9
NaN      1
Name: count, dtype: int64

### Imputing the null values


In [74]:


def knn_impute_numeric(df: pd.DataFrame, n_neighbors: int = 5) -> pd.DataFrame:
    
    df_copy = df.copy()
    
    numeric_cols = df_copy.select_dtypes(include=[float, int]).columns
    numeric_df = df_copy[numeric_cols]
    
    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputed_array = imputer.fit_transform(numeric_df)
    
    imputed_df = pd.DataFrame(imputed_array, columns=numeric_cols, index=df_copy.index)
    df_copy[numeric_cols] = imputed_df
    
    return df_copy


In [75]:
acquisitions = knn_impute_numeric(acquisitions)

acquisitions.isnull().sum()

Acquired Company                    0
Acquiring Company                   0
Year of acquisition announcement    0
Price                               0
Status                              0
Terms                               0
dtype: int64

In [76]:
acquisitions["Status"].value_counts(dropna=False)

Status
2.0    311
0.0     16
1.0      9
Name: count, dtype: int64